<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_LiveloEsfera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# 1. Instalação de pacotes necessários
!pip install selenium
!apt-get update # para o ChromeDriver
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [62]:
# 2. Importações
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

In [63]:
# 3. Configurações do Chrome headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

## Extração Livelo

In [75]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re

def extrair_beneficio_livelo(nome_loja, url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)

        # Espera até o benefício aparecer
        wait = WebDriverWait(driver, 15)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.css-1gdp5jf")))

        soup = BeautifulSoup(driver.page_source, "html.parser")

        span = soup.find("span", class_="css-1gdp5jf")
        texto = span.get_text(strip=True)

        # Ex.: "Ganhe até 2 pontos a cada 1 real gasto"
        match = re.search(r"até\s*(\d+)\s*ponto[s]?\s*a\s*cada\s*(\d+)\s*real", texto, re.IGNORECASE)
        if match:
            pts, reais = match.group(1), match.group(2)
            beneficio_format = f"{pts}x{reais} (Ganhe {pts} ponto{'s' if int(pts) > 1 else ''} a cada real)"
        else:
            beneficio_format = texto

        resultado = f"{nome_loja.title()} & Livelo = {beneficio_format}\n🔗 Link: {url} \n"

    except Exception as e:
        resultado = f"❌ Erro ao extrair dados da Livelo: {str(e)}"

    finally:
        driver.quit()

    return resultado

# Exemplos de uso:
livelo_amazon = extrair_beneficio_livelo("Amazon", "https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ")
livelo_gocase = extrair_beneficio_livelo("Gocase", "https://www.livelo.com.br/juntar-pontos/parceiros/gocase/GCS")

## Esfera unificado

In [72]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def extrair_beneficio_esfera(nome_loja, url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Verifica se tem estrutura Gocase (novos cards com fontes grandes)
        pontos_span = soup.find("span", class_="font-poppins leading-tight text-6xl font-semibold lg:mr-4 w-max")

        if pontos_span:
            # 🟢 Estrutura nova (ex: Gocase)
            pontos = pontos_span.text.strip()
            unidade = soup.find("span", class_="font-poppins leading-tight text-xl font-semibold tracking-tight text-boston").text.strip()
            programa = soup.find("span", class_="font-poppins leading-tight text-xl font-normal tracking-tight text-grey-darker").text.strip()

            if int(pontos) > 1:
                beneficio_formatado = f"{pontos}x1 (Ganhe {pontos} {unidade} {programa} a cada real)"
            else:
                beneficio_formatado = f"{pontos}x1 (Ganhe {pontos} {unidade[:-1]} {programa} a cada real)"

            resultado = f"{nome_loja} = {beneficio_formatado}\n🔗 Link: {url}\n"

        else:
            # 🟡 Estrutura antiga (ex: Amazon)
            titulo = soup.find("h1", class_="product-title").text.strip()
            beneficio_section = soup.find("p", string="Beneficio")
            beneficio_raw = beneficio_section.find_next("div").contents[0].strip()
            beneficio_info = beneficio_section.find_next("div").find("p").text.strip()
            validade = soup.find("p", string="Validade").find_next("div").text.strip()
            credito = soup.find("p", string="Creditado em").find_next("div").text.strip()

            resultado = f"{titulo} = {beneficio_raw} ({beneficio_info})\n🔗 Link: {url}\n"

    except Exception as e:
        resultado = f"❌ Erro ao extrair dados da Esfera: {str(e)}"
    finally:
        driver.quit()

    return resultado.strip()

esfera_amazon = extrair_beneficio_esfera("Amazon", "https://www.esfera.com.vc/p/amazon/e000100298")
esfera_gocase = extrair_beneficio_esfera("Gocase", "https://www.esfera.com.vc/p/gocase/e000100705")

## Unificando os dados

In [80]:
df_final = f"""🔴 Esfera & {esfera_amazon}

🟣 Livelo & {livelo_amazon}

------------------------------------

🔴 Esfera & {esfera_gocase}

🟣 Livelo & {livelo_gocase}

"""

In [81]:
print(df_final)

🔴 Esfera & Amazon = 1x1 (Ganhe 1 ponto a cada real)
🔗 Link: https://www.esfera.com.vc/p/amazon/e000100298

🟣 Livelo & Amazon & Livelo = 1x1 (Ganhe 1 ponto a cada real)
🔗 Link: https://www.livelo.com.br/juntar-pontos/parceiros/amazon/AMZ 


------------------------------------

🔴 Esfera & Gocase = 2x1 (Ganhe 2 Pontos Esfera* a cada real)
🔗 Link: https://www.esfera.com.vc/p/gocase/e000100705

🟣 Livelo & Gocase & Livelo = 2x1 (Ganhe 2 pontos a cada real)
🔗 Link: https://www.livelo.com.br/juntar-pontos/parceiros/gocase/GCS 





## Enviar E-mail

In [70]:
#importações
from email.message import EmailMessage
import os
import smtplib

#definindo dados de envio do e-mail
remetente = os.environ['EMAIL_REMETENTE']
destinatario = os.environ['EMAIL_DESTINATARIO']
assunto = 'Seu resumo diário pontos Amazon'
mensagem = f"""Olá, aqui está o seu resumo dos pontos Esfera e Livelo:

------------------------------------

{df_final}
"""

senha = os.environ['SENHA_GMAIL']

#crinado o e-mail
msg = EmailMessage()
msg['Subject'] = assunto
msg['From'] = remetente
msg['To'] = destinatario
msg.set_content(mensagem)

#enviando o e-mail
with smtplib.SMTP_SSL("smtp.gmail.com",465) as email:
  email.login(remetente,senha)
  email.send_message(msg)

print("O email foi enviado com sucesso!")


KeyError: 'EMAIL_REMETENTE'